In [24]:
import os
os.chdir("..")

In [ ]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'DevEstate.settings')
# This is for async, in case we are going to see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [5]:
# WORKING API CALL

#Import Necessities: SSL is a Temporary fix for a handshake bug, will fix later
import http.client
import ssl
import time
import json
import secrets
ssl._create_default_https_context = ssl._create_unverified_context

# Returns a string of JSON Information regarding every property in New York City
def fetch_all_data():
   conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
   headers = {
        'X-RapidAPI-Key': secrets.key,
        'X-RapidAPI-Host': secrets.host
   }
   conn.request("GET", "/searchByUrl?url=https://www.zillow.com/new-york-ny/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-74.15820883203124%2C%22east%22%3A-73.66382406640624%2C%22south%22%3A40.57279370530749%2C%22north%22%3A40.93585324525284%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D", headers=headers)
   res = conn.getresponse()
   data = res.read()
   return(data.decode("utf-8"))

#Converting that string into a Dictionary
listingsInfo = json.loads(fetch_all_data()) #ALL data is now in a dictionary

#Returns a variety of information on a property based on its unique ZPID (Zillow ID)
def fetch_individual_data(ZPID):
    conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': secrets.key,
        'X-RapidAPI-Host': secrets.host
    }
    conn.request("GET", "/property?zpid=" + str(ZPID), headers=headers)
    res = conn.getresponse()
    data = res.read()
    conn.close()  # Close the connection after the request
    return data.decode("utf-8")

#Parses through a Dictionary of Listings in order to obtain each one's ZPID; Then, fetches the data associated with each one
def parseListings(listingDataDictionary):
    listingDataDictionaryList = []
    props = listingDataDictionary.get("props", [])
    
    for i, prop in enumerate(props):
        try:
            ZPID = prop.get("zpid")
            if ZPID:  # Only proceed if ZPID is present
                print(f"Fetching data for ZPID: {ZPID}")
                individual_property_data = fetch_individual_data(ZPID)
                listingDataDictionaryList.append(json.loads(individual_property_data))
                time.sleep(1)  # Sleep for 1 second between API calls to avoid rate limiting
            else:
                print(f"No ZPID found for property at index {i}")
        except Exception as e:
            print(f"Error fetching data for property at index {i}: {e}")
            # Optionally, break or continue depending on whether you want to stop at the first error or keep going
            # break

    return listingDataDictionaryList
print(parseListings(listingsInfo))


Fetching data for ZPID: 32209658
Fetching data for ZPID: 321911266
Fetching data for ZPID: 2056637647
Fetching data for ZPID: 2067603580
Fetching data for ZPID: 68310361
Fetching data for ZPID: 244753088
Fetching data for ZPID: 215955203
Fetching data for ZPID: 2067535592
Fetching data for ZPID: 31534257
Fetching data for ZPID: 2054468761
Fetching data for ZPID: 32197678
Fetching data for ZPID: 2064145239
Fetching data for ZPID: 2069146930
Fetching data for ZPID: 29826190
Fetching data for ZPID: 83054127
Fetching data for ZPID: 2053192637
Fetching data for ZPID: 245370034
Fetching data for ZPID: 2085844247
Fetching data for ZPID: 245026728
Fetching data for ZPID: 30768564
Fetching data for ZPID: 32021655
Fetching data for ZPID: 2062695224
Fetching data for ZPID: 2111004401
Fetching data for ZPID: 79727211
Fetching data for ZPID: 119908765
Fetching data for ZPID: 32284423
Fetching data for ZPID: 244701648
Fetching data for ZPID: 61969351
Fetching data for ZPID: 244428185
Fetching data f